In [1]:
import pandas as pd

## importing the dataset and printing

In [58]:
europe_df = pd.read_csv("dataset.csv")
europe_df.head(4)

C:\Users\BT\AppData\Local\Temp\ipykernel_15564\2931737287.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  europe_df = pd.read_csv("dataset.csv")


,restaurant_link,restaurant_name,original_location,country,region,province,city,address,latitude,longitude,...,excellent,very_good,average,poor,terrible,food,service,value,atmosphere,keywords
0,g10001637-d10002227,Le 147,"[""Europe"", ""France"", ""Nouvelle-Aquitaine"", ""Ha...",France,Nouvelle-Aquitaine,Haute-Vienne,Saint-Jouvent,"10 Maison Neuve, 87510 Saint-Jouvent France",45.961674,1.169131,...,2.0,0.0,0.0,0.0,0.0,4.0,4.5,4.0,NaN,NaN
1,g10001637-d14975787,Le Saint Jouvent,"[""Europe"", ""France"", ""Nouvelle-Aquitaine"", ""Ha...",France,Nouvelle-Aquitaine,Haute-Vienne,Saint-Jouvent,"16 Place de l Eglise, 87510 Saint-Jouvent France",45.957040,1.205480,...,2.0,2.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2,g10002858-d4586832,Au Bout du Pont,"[""Europe"", ""France"", ""Centre-Val de Loire"", ""B...",France,Centre-Val de Loire,Berry,Rivarennes,"2 rue des Dames, 36800 Rivarennes France",46.635895,1.386133,...,3.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3,g10002986-d3510044,Le Relais de Naiade,"[""Europe"", ""France"", ""Nouvelle-Aquitaine"", ""Co...",France,Nouvelle-Aquitaine,Correze,Lacelle,"9 avenue Porte de la Correze 19170, 19170 Lace...",45.642610,1.824460,...,1.0,0.0,0.0,0.0,0.0,4.5,4.5,4.5,NaN,NaN


Let's print all the columns with mixed values

### Checking for the columns with mixed values

In [60]:

for col in europe_df.columns:
    weird = (europe_df[[col]].applymap(type) != europe_df[[col]].iloc[0].apply(type)).any(axis=1)
    if len(europe_df[weird]) > 0:
        print(col)

region
province
city
claimed
awards
popularity_detailed
popularity_generic
top_tags
price_level
price_range
meals
cuisines
special_diets
features
original_open_hours
default_language
keywords


In [61]:
# Let's count nan values in each of these columns
print("Length of dataset is", len(europe_df.isna().dropna()))

Length of dataset is 1083397


In [62]:
# Droping the Empty cells in the dataset
europe_df=europe_df.dropna()

In [63]:
# Checking for mixed values columns again
for col in europe_df.columns:
    weird = (europe_df[[col]].applymap(type) != europe_df[[col]].iloc[0].apply(type)).any(axis=1)
    if len(europe_df[weird]) > 0:
        print(col)

There is no mixed values columns remaining. Hence, removing the empty values removed the problem of mixed values

### correlation matrix

In [65]:
import matplotlib.pyplot as plt

corr = europe_df.corr()

corr.style.background_gradient(cmap='coolwarm')


,latitude,longitude,open_days_per_week,open_hours_per_week,working_shifts_per_week,avg_rating,total_reviews_count,reviews_count_in_default_language,excellent,very_good,average,poor,terrible,food,service,value,atmosphere
latitude,1.000000,-0.167626,0.001454,-0.119267,0.006026,-0.117522,-0.190646,0.117431,0.064718,0.151371,0.174668,0.224043,0.203757,-0.116431,-0.124475,-0.173726,-0.237497
longitude,-0.167626,1.000000,0.147847,0.257508,0.002663,0.060358,0.004178,-0.133565,-0.102032,-0.165313,-0.139217,-0.138359,-0.129623,0.045573,0.068223,0.094983,0.126168
open_days_per_week,0.001454,0.147847,1.000000,0.551182,0.385544,-0.204520,0.035785,0.039965,-0.020050,0.111339,0.122652,0.134315,0.148962,-0.227055,-0.185077,-0.173202,-0.101240
open_hours_per_week,-0.119267,0.257508,0.551182,1.000000,-0.010086,-0.223372,0.037120,0.020474,-0.041137,0.098145,0.107368,0.114844,0.149115,-0.241131,-0.195055,-0.148055,-0.059487
working_shifts_per_week,0.006026,0.002663,0.385544,-0.010086,1.000000,-0.065222,0.085144,-0.000887,-0.009158,0.009612,0.017264,0.016327,0.003992,-0.057071,-0.058868,-0.111367,-0.082318
avg_rating,-0.117522,0.060358,-0.204520,-0.223372,-0.065222,1.000000,0.028484,-0.002495,0.173696,-0.180352,-0.307421,-0.391741,-0.451994,0.746088,0.751317,0.644715,0.470596
total_reviews_count,-0.190646,0.004178,0.035785,0.037120,0.085144,0.028484,1.000000,0.792266,0.781435,0.723794,0.622775,0.494250,0.434019,0.039960,-0.006927,-0.073920,0.097625
reviews_count_in_default_language,0.117431,-0.133565,0.039965,0.020474,-0.000887,-0.002495,0.792266,1.000000,0.958827,0.922048,0.846087,0.731350,0.637788,0.012198,-0.019469,-0.096617,0.056696
excellent,0.064718,-0.102032,-0.020050,-0.041137,-0.009158,0.173696,0.781435,0.958827,1.000000,0.790189,0.672573,0.538382,0.448761,0.172486,0.148564,0.065835,0.167678
very_good,0.151371,-0.165313,0.111339,0.098145,0.009612,-0.180352,0.723794,0.922048,0.790189,1.000000,0.914253,0.799027,0.700986,-0.156339,-0.186690,-0.244713,-0.062866


There are dark colors in the axis where rating and review columns are. So, Yes, review columns are correlated with rating columns

### Vegetarian vs Non Veg

In [66]:
veg_non_veg = europe_df.groupby("vegetarian_friendly")["avg_rating"].mean()

In [67]:
veg_non_veg

vegetarian_friendly
N    4.368421
Y    4.341745
Name: avg_rating, dtype: float64

There is no apparent different between average rating of vegetarian and non-veg restaurants.

### Significantly Expensive Cousines

In [68]:
def parse_price(price_range):
    high_value = price_range.split('-')[1]
    if 'CHF' in high_value:
        high_value = high_value[4:]
    else:
        high_value = high_value[1:]
        
    return int(high_value.replace(',', ''))


In [73]:
# creating a dataframe for cuisines and price_range
cuisines_price_df = europe_df[europe_df['price_range'].notna()][['price_range', 'cuisines']]

# creating a max_price column
cuisines_price_df['max_price'] = cuisines_price_df['price_range'].apply(parse_price)

# displaying the cuisines_price dataframe by sorting the dataframe by max_price in descending order
cuisines_price_df.sort_values(by='max_price', ascending=False).head(5)

,price_range,cuisines,max_price
630349,"€1,149-€2,880","Bar, British, Pub",2880
376546,€10-€999,"Steakhouse, Barbecue, Argentinian",999
1063708,€125-€664,"French, Central European, Danish",664
463797,€32-€296,Healthy,296
878443,€80-€200,"Steakhouse, Italian, Seafood, Mediterranean",200


### Opening a Gluten Free Restaurant

In [74]:
new_df = pd.read_csv("europe_capitals_population_and_area.csv", delimiter=";")
new_df.head(4)

,CityNo,City,Country,Population,Area in km2
0,1,Amsterdam,Netherland,851573,"219,32"
1,2,Athens,Greece,664046,2929
2,4,Barcelona,Spain,1620809,"101,4"
3,4,Berlin,Germany,3711930,"891,7"


In [91]:
europe_df = pd.read_csv("dataset.csv")

C:\Users\BT\AppData\Local\Temp\ipykernel_15564\236892588.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  europe_df = pd.read_csv("dataset.csv")


In [102]:
europe_df_gluten_free = europe_df[europe_df["gluten_free"] == "Y"].groupby("city")["restaurant_name"].count()
europe_df_gluten_free.sort_values().head(5)

city
Lascazeres    1
Luebbenau     1
Ludvika       1
Ludham        1
Ludgvan       1
Name: restaurant_name, dtype: int64

In [105]:
# adding a new column to hold count of gluten_free restaurants for each capital using gluten_free_count created above
new_df['gluten_free_restaurants'] = new_df['City'].apply(lambda x: europe_df_gluten_free[x] if x in europe_df_gluten_free else 0)

In [109]:
new_df["density"] = new_df["gluten_free_restaurants"]/new_df["Population"]

In [111]:
new_df.sort_values(by="density").head(5)

,CityNo,City,Country,Population,Area in km2,gluten_free_restaurants,density
15,16,Ljubljana,Slovenia,279631,"163,8",0,0.0
23,24,Oslo,Norway,634293,454,0,0.0
22,23,Oporto,Portugal,214349,"41,42",0,0.0
17,18,Luxembourg,Luxembourg,590667,"2586,4",0,0.0
16,17,London,England,8825000,607,0,0.0


In [112]:
europe_df.groupby("cuisines")["avg_rating"].mean().sort_values(ascending=False)

cuisines
Italian, Barbecue, Grill, Argentinian, South American     5.0
Italian, Mediterranean, International, Grill, Wine Bar    5.0
Chinese, Fast food, Asian, Thai, British                  5.0
Bar, Spanish, French, European                            5.0
Greek, Albanian, European                                 5.0
                                                         ... 
Vietnamese, Eastern European                              NaN
Wine Bar, Apulian, Southern-Italian                       NaN
Wine Bar, Dining bars, French, Bar, Pub, Diner            NaN
Wine Bar, Middle Eastern, Persian                         NaN
Wine Bar, Southern-Italian                                NaN
Name: avg_rating, Length: 97741, dtype: float64

So, we can see that european and scandinavian cuisines have the most rating. So, if you want to open a new restaurant, it is better to open in these cuisines.